1 define path

In [3]:
import os
from glob import glob

In [ ]:
#### Eckert 4 sources ####

# homepath = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/data/ID828/reduction'
# homepath = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/data/ID9647/reduction'
homepath = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/data/ID3460/reduction/SDSSTG3460'
# homepath = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/data/RGH80/reduction'
os.chdir(homepath)

2 Initialize steps

In [14]:
#### alldata ####
homepaths = glob(f'/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/SDSS*')
for homepath in homepaths:
    os.chdir(homepath)
    SRCNAME2 = homepath.split('/')[-1]
    f = open('cifbuild.sh', 'w')
    f.write(
    f'''
    export SAS_ODF={homepath}
    cifbuild 2>&1 | tee cifbuild.log

    '''
    )
    f.close()


In [15]:
#### alldata ####
homepaths = glob(f'/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/SDSS*')
for homepath in homepaths:
    os.chdir(homepath)
    f = open('odfingest.sh', 'w')
    f.write(f'''
export SAS_CCF={homepath}/ccf.cif
export SAS_ODF={homepath}
odfingest odfdir=$SAS_ODF outdir=$SAS_ODF 2>&1 | tee odfingest.log
    ''')
    f.close()

3 Set SAS environment

In [31]:
from glob import glob
import os

homepaths = glob('/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/SDSSTG2620/*0*')
i = 0

for homepath in homepaths:
    os.chdir(homepath)
    f = open('set_sas.txt', 'w')
    
    # Check if there are any files matching the pattern *SUM.SAS
    matching_files = glob('*SUM.SAS')
    
    if matching_files:
        sasfile = matching_files[0]
        f.write(f'''
export SAS_CCF={homepath}/ccf.cif
export SAS_ODF={homepath}/{sasfile}
sasversion
        ''')
        f.close()
    else:
        i += 1
        print(homepath)

print(f"Number of directories without *SUM.SAS files: {i}")


Number of directories without *SUM.SAS files: 0


4 emproc

In [30]:
from glob import glob
import os

homepaths = glob('/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/SDSSTG2620/*0*')

for homepath in homepaths:
    os.chdir(homepath)

    # Check if there are any files matching the pattern *SUM.SAS
    matching_files = glob('*SUM.SAS')

    if matching_files:
        # Open emproc.sh and write the necessary content
        with open('emproc.sh', 'w') as f_emproc:
            f_emproc.write(f'''
source {homepath}/set_sas.txt
emproc 2>&1 | tee emproc.log

# mv *EMOS1_*_ImagingEvts.ds mos1-S001.evt
# mv *EMOS2_*_ImagingEvts.ds mos2-S002.evt
            ''')

        # Make emproc.sh executable
        os.system('chmod +x emproc.sh')



5 epproc

In [29]:
from glob import glob
import os

homepaths = glob('/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/SDSSTG2620/*0*')

for homepath in homepaths:
    os.chdir(homepath)

    # Check if there are any files matching the pattern *SUM.SAS
    matching_files = glob('*SUM.SAS')

    if matching_files:
        f = open('epproc.sh', 'w')
        f.write(
        '''
source set_sas.txt
epproc 2>&1 |tee epproc.log
mv *EPN*ImagingEvts.ds pn-S003.evt
epproc withoutoftime=true 2>&1 |tee epproc_oot.log
mv *EPN*ImagingEvts.ds pn-oot-S003.evt
        '''
        )
        f.close()

        # Make epproc.sh executable
        os.system('chmod +x epproc.sh')


6 emanom: diagnose abnormal bright chips in MOS
NO NEED! the esas has run the *-anom.log!

In [41]:
from glob import glob
import os

homepaths = glob('/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/SDSS*')
for homepath in homepaths:
    if os.path.isdir(homepath):
        os.chdir(homepath)
        f = open('emanom.sh', 'w')
        f.write(
'''
emanom eventfile=mos1-S001.evt keepcorner=no 2>&1 | tee emanom_mos1.log
emanom eventfile=mos2-S002.evt keepcorner=no 2>&1 | tee emanom_mos2.log
'''
        )
        f.close()

    # mos2 chip5 is I, but not serious by visually inspect, not excluded. 
    # This observation is indeed contaminated by SP. (40% exp time is flared)


In [43]:
##### automatic one ######
# if the chip is B or O, set chip to F
homepaths = glob('/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/SDSS*')

for homepath in homepaths:
    if os.path.isdir(homepath):
        os.chdir(homepath)
        # Initialize M1ON and M2ON
        output_dir = {'mos1':["T", "T", "T", "T", "T", "T", "T"], 'mos2':["T", "T", "T", "T", "T", "T", "T"]}
        for inst in ['mos1', 'mos2']:
            with open(f'emanom_{inst}.log') as f:

                # Parse the output of emanom
                for line in f.readlines():
                    # Extract CCD index, status, and M1/M2 information from the line
                    parts = line.split()
                    is_m1 = "CCD:" in line
                    
                    # Check if the status is 'O' or 'B'
                    if is_m1:
                        ccd_index = int(parts[3])
                        status = parts[10]
                        if status in ["O", "B"]:
                            # Set the corresponding CCD index of M1ON or M2ON as 'F'
                            output_dir[inst][ccd_index - 1] = "F"
                    
            # Output the resulting export commands
            with open('set_chips_on.txt', 'w') as newf:
                newf.write(f'''
export M1ON="{' '.join(output_dir['mos1'])}"
export M2ON="{' '.join(output_dir['mos2'])}"
export PNON="T T T T"
''')

                


In [43]:
##### manual one #####
f = open('set_chips_on.txt', 'w')
f.write(
'''
export M1ON="T T F F T F T"
export M2ON="T T T T F T T"
export PNON="T T T T"
'''
)
f.close()

In [42]:
# visual inspect
## NOTE: The abnormal state of mos1 chip4 can't be seen from the image!
homepaths = glob('/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/SDSS*')

for homepath in homepaths:
    if os.path.isdir(homepath):
        os.chdir(homepath)
        f = open('chips_visual_inspect.sh', 'w')
        f.write(
'''
evselect table=mos1-S001.evt withimageset=yes imageset=mos1S001-diag-det-unfilt.fits filtertype=expression expression="(PI in [300:1000])&&(PATTERN<=12)&&((FLAG & 0x766aa000)==0)" ignorelegallimits=yes imagebinning=imageSize xcolumn=DETX ximagesize=780 ximagemax=19500 ximagemin=-19499 ycolumn=DETY yimagesize=780 yimagemax=19500 yimagemin=-19499
evselect table=mos2-S002.evt withimageset=yes imageset=mos2S002-diag-det-unfilt.fits filtertype=expression expression="(PI in [300:1000])&& (PATTERN<=12)&&((FLAG & 0x766aa000)==0)" ignorelegallimits=yes imagebinning=imageSize xcolumn=DETX ximagesize=780 ximagemax=19500 ximagemin=-19499 ycolumn=DETY yimagesize=780 yimagemax=19500 yimagemin=-19499
evselect table=pn-S003.evt withimageset=yes imageset=pnS003-diag-det-unfilt.fits filtertype=expression expression="(PI in [300:1000])&& (PATTERN <= 4)&&(#XMMEA_EP)" ignorelegallimits=yes imagebinning=imageSize xcolumn=DETX ximagesize=780 ximagemax=19500 ximagemin=-19499 ycolumn=DETY yimagesize=780 yimagemax=19500 yimagemin=-19499
evselect table=pn-oot-S003.evt withimageset=yes imageset=pnS003-oot-diag-det-unfilt.fits filtertype=expression expression="(PI in [300:1000])&& (PATTERN <= 4)&&(#XMMEA_EP)" ignorelegallimits=yes imagebinning=imageSize xcolumn=DETX ximagesize=780 ximagemax=19500 ximagemin=-19499 ycolumn=DETY yimagesize=780 yimagemax=19500 yimagemin=-19499
'''
        )
        f.close()


APPENDIX check the image of event files
NOTE: ds9 event files and ds9 image extract from event files are not the same!

In [51]:
# visual inspect
## NOTE: The abnormal state of mos1 chip4 can't be seen from the image!
homepaths = glob('/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/SDSSTG2620')

for homepath in homepaths:
    if os.path.isdir(homepath):
        os.chdir(homepath)
        print(homepath)
        with open('ext_img.sh', 'w') as f:
            for evtfile in glob('*-exchips.fits'):
                print(evtfile)
                f.write(
f'''
evselect table={evtfile} withimageset=yes imageset={evtfile}_img.fits filtertype=expression expression="(PI in [300:1000])&&(PATTERN<=12)&&((FLAG & 0x766aa000)==0)" ignorelegallimits=yes imagebinning=imageSize xcolumn=DETX ximagesize=780 ximagemax=19500 ximagemin=-19499 ycolumn=DETY yimagesize=780 yimagemax=19500 yimagemin=-19499
'''
        )


/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/SDSSTG2620
pn-allevc-exchips.fits
pnS003-allevcoot-exchips.fits
mos1-allevc-exchips.fits
mos2-allevc-exchips.fits


7 SP flare filtering

In [46]:
homepaths = glob('/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP/SDSS*')

for homepath in homepaths:
    if os.path.isdir(homepath):
        os.chdir(homepath)
        f = open('sp_flare.sh', 'w')
        f.write(
'''
rm *allevc*fits
espfilt eventfile=mos1-S001.evt elow=2500 ehigh=8500 withsmoothing=yes smooth=51 rangescale=6.0 allowsigma=3.0 method=histogram keepinterfiles=false 2>&1 | tee flare_filter_mos1-S001.log
espfilt eventfile=mos2-S002.evt elow=2500 ehigh=8500 withsmoothing=yes smooth=51 rangescale=6.0 allowsigma=3.0 method=histogram keepinterfiles=false 2>&1 | tee flare_filter_mos2-S002.log
espfilt eventfile=pn-S003.evt elow=2500 ehigh=8500 withsmoothing=yes smooth=51 rangescale=15.0 allowsigma=3.0 method=histogram withoot=Y ootfile=pn-oot-S003.evt keepinterfiles=false 2>&1 | tee flare_filter_pn-S003.log
'''
        )

        f.close()
        # except for P-allevc.fits and P-gti.fits are kept, others are moved to DIAGNOISE dir after checking
##!!!! NOTE: output the *allevc*fits files!

APPENDIX Merge diff obs if they point the same but on diff orbit
(see p67 cookbook)


In [ ]:
"""
1. atthkgen
2. merge the events and the atthks
NOTE: 
1. Make sure the set1 is observed before set2!
2. Make sure their pointing are close enough!
"""


source set_sas.txt
rm atthkgen.log
atthkgen 2>&1 | tee -a atthkgen.log

merge set1=0091140401/atthk.dat set2=0841481301/atthk.dat outset=atthk.dat
merge set1=0091140401/atthk.dat set2=0841481301/atthk.dat outset=atthk.dat

merge set1=0091140401/mos1S002-allevc.fits set2=0841481301/mos1S001-allevc.fits outset=mos1S001-allevc.fits mergedifferentobs="yes"
merge set1=0091140401/mos2S003-allevc.fits set2=0841481301/mos2S002-allevc.fits outset=mos2S002-allevc.fits mergedifferentobs="yes"
merge set1=0091140401/pnS005-allevc.fits set2=0841481301/pnS003-allevc.fits outset=pnS003-allevc.fits mergedifferentobs="yes"
merge set1=0091140401/pnS005-allevcoot.fits set2=0841481301/pnS003-allevcoot.fits outset=pnS003-allevcoot.fits mergedifferentobs="yes"

for f in *allevc*
do
evselect table=$f withimageset=yes imageset=$f.img.fits filtertype=expression expression="(PI in [300:1000])&&(PATTERN<=12)&&((FLAG & 0x766aa000)==0)" ignorelegallimits=yes imagebinning=imageSize xcolumn=DETX ximagesize=780 ximagemax=19500 ximagemin=-19499 ycolumn=DETY yimagesize=780 yimagemax=19500 yimagemin=-19499

done

8 check sp flare filtering diagnostic files
Files: P-hist.qdp
Conclusion: 
not very flared


9 Remove the point sources: go to pointsources-pipeline.ipynb

10 Convert the annuli region from fk5 coords to detXY

In [ ]:
from astropy.coordinates import SkyCoord
import astropy.units as u

def deg2sex(ra_deg, dec_deg):
    # Create a SkyCoord object with the specified units
    coords = SkyCoord(ra=ra_deg * u.degree, dec=dec_deg * u.degree)
    # Extract the converted sexagesimal values
    ra_sexagesimal = coords.to_string(style='hmsdms', precision=2)
    return f"{ra_sexagesimal.split(' ')[0]}", f"{ra_sexagesimal.split(' ')[1]}"

In [ ]:
# convert the center coords to det XY

# read the fk5 eckert region
# datapath = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/data/RGH80/eckert/0105860101/region'
# datapath = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/data/ID828/eckert/ID828/0904720501/regions'
# datapath = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/data/ID9647/eckert/ID9647/0904720201/regions'
datapath = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/data/ID9647/eckert/ID9647/0904720201/regions'

file = f'{datapath}/SDSSTG9647_reg0.reg'
f = open(file)
line = f.readlines()[-1]
ra = line.split('(')[-1].split(',')[0]
dec = line.split(',')[1]
f.close()

# write the command to convert 
f = open('deg2det.sh', 'w')

f.write(
f'''
source set_sas.txt
export M1=mos1S001
export M2=mos2S002
export PN=pnS003

esky2det datastyle=user ra={ra} dec={dec} outunit=det calinfoset='mos1S001-allevc-exchips.fits' 2>&1 | tee deg2det-$M1.log
esky2det datastyle=user ra={ra} dec={dec} outunit=det calinfoset='mos2S002-allevc-exchips.fits' 2>&1 | tee deg2det-$M2.log
esky2det datastyle=user ra={ra} dec={dec} outunit=det calinfoset='pnS003-allevc-exchips.fits' 2>&1 | tee deg2det-$PN.log
''')

f.close()


In [ ]:
''' RGH 80
   307.6     -183.9
   15.1       90.4
   132.2     -235.2

'''

detx_dict = {'mos1S001':307.6, 'mos2S002':15.1, 'pnS003':132.2}
dety_dict = {'mos1S001':-183.9, 'mos2S002':90.4, 'pnS003':-235.2}

In [ ]:
print(os.getcwd())
'''ID3460
294.8     -203.6
34.7       77.5
112.5     -222.4


'''

detx_dict = {'mos1S001':294.8 , 'mos2S002':34.7, 'pnS003':112.5}
dety_dict = {'mos1S001':-203.6, 'mos2S002':77.5, 'pnS003':-222.4}

In [ ]:
# read the fk5 eckert region
from glob import glob
import numpy as np

# datapath = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/data/ID3460/eckert/ID3460/0903892401/regions'
# datapath = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/data/ID9647/eckert/ID9647/0904720201/regions'
datapath = glob(f'/Users/eusracenorth/Documents/work/XGAP-ABUN/data/RGH80/eckert/*/regions')[0]
files = glob(f'{datapath}/RGH80_reg*.reg')

reg_radius = []
for file in files:
    f = open(file)
    line = f.readlines()[-1]
    reg_radius.append(float(line.split(',')[-1].split("'")[0]))
    f.close()

reg_radius = np.unique(reg_radius)
print(reg_radius)

In [ ]:
## create the region files

# detx_dict = {'mos1S001':255.7, 'mos2S002':70.1, 'pnS003':76.8}
# dety_dict = {'mos1S001':-239.1, 'mos2S002':38.3, 'pnS003':-183.5}

# os.makedirs(f'{homepath}/spec_reg')
os.chdir(f'{homepath}/spec_reg')

r500 = 7.35216
bkg_r2 = 14.799


for name in ['mos1S001', 'mos2S002', 'pnS003']: # 
    for i in range(1, len(reg_radius)):
        f = open(f'reg{i}-{name}.reg', 'w')
        f.write(
f'''&&((DETX,DETY) IN annulus({detx_dict[name]},{dety_dict[name]},{(reg_radius[int(i-1)] * 60/0.05):.2f},{(reg_radius[i]* 60/0.05):.2f}))''')
    
        f.close()

    f = open(f'reg0-{name}.reg', 'w')
    f.write(
f'''&&((DETX,DETY) IN circle({detx_dict[name]},{dety_dict[name]},{(reg_radius[0]*60/0.05):.2f}))
''')
    f.close()

    f = open(f'reg{len(reg_radius)}-{name}.reg', 'w')
    f.write(
f'''&&((DETX,DETY) IN circle({detx_dict[name]},{dety_dict[name]},{(reg_radius[-1]*60/0.05):.2f}))
''')
    f.close()

    f = open(f'R500-{name}.reg', 'w')
    f.write(
f'''&&((DETX,DETY) IN circle({detx_dict[name]},{dety_dict[name]},{(r500*60/0.05):.2f}))
''')
    f.close()

    f = open(f'bkg-{name}.reg', 'w')
    f.write(
f'''&&((DETX,DETY) IN annulus({detx_dict[name]},{dety_dict[name]},{(reg_radius[-1]*60/0.05):.2f},{(bkg_r2*60/0.05):.2f}))
''')
    f.close()





13 mosspectra mosback

In [ ]:
# os.chdir(homepath)

f = open('mosspectra_reg_rest.sh', 'w')
f.write(
'''
source set_chips_on.txt
source set_sas.txt

export ELO=700
export EHI=1200
export M1=mos1S001
export M2=mos2S002

for regfile in spec_reg/*-mos1S001.reg
do
regname=$(basename $regfile | cut -d'-' -f 1)

echo $regname

mkdir ${regname}
mkdir ${regname}/img
mkdir ${regname}/logs
mkdir ${regname}/diagnose
mkdir ${regname}/mos_spec
mkdir ${regname}/fitting

rm ${regname}/logs/mosspectra_${M1}.log
rm ${regname}/logs/mosspectra_${M2}.log

mosspectra eventfile=${M1}-allevc.fits keepinterfiles=yes withregion=yes regionfile=spec_reg/${regname}-${M1}.reg pattern=12 withsrcrem=yes maskdet=${M1}-bkgregtdet.fits masksky=${M1}-bkgregtsky.fits elow=${ELO} ehigh=${EHI} ccds="${M1ON}" -V=7  >> ${regname}/logs/mosspectra_${M1}.log 2>&1
mosback inspecfile=${M1}-fovt.pi elow=${ELO} ehigh=${EHI} ccds="${M1ON}" >> mosback_${M1}.log 2>&1 
mosspectra eventfile=${M2}-allevc.fits keepinterfiles=yes withregion=yes regionfile=spec_reg/${regname}-${M1}.reg pattern=12 withsrcrem=yes maskdet=${M2}-bkgregtdet.fits masksky=${M2}-bkgregtsky.fits elow=${ELO} ehigh=${EHI} ccds="${M2ON}" -V=7 >> ${regname}/logs/mosspectra_${M2}.log 2>&1
mosback inspecfile=${M2}-fovt.pi elow=${ELO} ehigh=${EHI} ccds="${M2ON}" >> mosback_${M2}.log 2>&1

for name in $M1 $M2
do
echo $name

mv ${name}*-${ELO}-${EHI}* ${regname}/img
mv ${name}*.pi ${regname}/mos_spec
mv ${name}*.qdp ${regname}/diagnose
mv ${name}*imt* ${regname}/img
mv ${name}*.arf ${regname}/fitting
mv ${name}*.rmf ${regname}/fitting
mv ${name}*imspdet* ${regname}/img

cp ${regname}/mos_spec/${name}-fovt.pi ${regname}/fitting
cp ${regname}/mos_spec/${name}-bkg.pi ${regname}/fitting

done
done
'''
)
f.close()

14 pnspectra, pnback

In [ ]:
os.chdir('/Users/eusracenorth/Documents/work/XGAP-ABUN/codes/ESAS/tmp')
f = open('pnspectra_01r500.sh', 'w')

f.write('''
source set_chips_on.txt
source set_sas.txt

export PN=pnS003
export ELO=700
export EHI=1200

for pat in 0 4
do
for regfile in spec_reg/R500-01-mos1S001.reg
do
regname=$(basename $regfile | cut -d'-' -f 1)
echo $PN
echo $regname

mkdir ${regname}
mkdir ${regname}/img
mkdir ${regname}/logs
mkdir ${regname}/diagnose
mkdir ${regname}/mos_spec
mkdir ${regname}/fitting

rm ${regname}/logs/pnspectra_${pat}_${PN}.log
rm ${regname}/logs/pnback_${pat}_${PN}.log

pnspectra eventfile=${PN}-allevc.fits ootevtfile=${PN}-allevcoot.fits keepinterfiles=yes withregion=yes regionfile=spec_reg/${regname}-${PN}.reg pattern=${pat} withsrcrem=yes maskdet=${PN}-bkgregtdet.fits masksky=${PN}-bkgregtsky.fits elow=${ELO} ehigh=${EHI} quads="${PNON}" badpixelresolution=1 -V=7 2>&1 | tee  ${regname}/logs/pnspectra_${pat}_${PN}.log
pnback inspecfile=${PN}-fovt.pi inspecoot=${PN}-fovtoot.pi elow=${ELO} ehigh=${EHI} quads="${PNON}" 2>&1 | tee ${regname}/logs/pnback_${pat}_${PN}.log 

rm -r ${regname}/${PN}_${pat}
mkdir ${regname}/${PN}_${pat}

mv pn*-${ELO}-${EHI}* ${regname}/${PN}_${pat}
mv pn*.pi ${regname}/${PN}_${pat}
mv pn*.qdp ${regname}/${PN}_${pat}
mv pn*imt* ${regname}/${PN}_${pat}
mv pn*.arf ${regname}/${PN}_${pat}
mv pn*.rmf ${regname}/${PN}_${pat}
mv pn*imspdet* ${regname}/${PN}_${pat}


cp ${regname}/${PN}_${pat}/${PN}-fovt.pi ${regname}/fitting/${PN}-${pat}-fovt.pi
cp ${regname}/${PN}_${pat}/${PN}-fovtoot.pi ${regname}/fitting/${PN}-${pat}-fovtoot.pi
cp ${regname}/${PN}_${pat}/${PN}-bkg.pi ${regname}/fitting/${PN}-${pat}-bkg.pi
cp ${regname}/${PN}_${pat}/${PN}.rmf ${regname}/fitting/${PN}-${pat}.rmf
cp ${regname}/${PN}_${pat}/${PN}.arf ${regname}/fitting/${PN}-${pat}.arf

done
done
'''
)
f.close()

In [ ]:
os.chdir(f'/Users/eusracenorth/Documents/work/XGAP-ABUN/data/ID3460/reduction/SDSSTG3460/reg0/pnS003_0/spec')

f = open('pnback_sup.sh', 'w')

f.write('''
source ../../../set_chips_on.txt
source ../../../set_sas.txt
export PN=pnS003
export ELO=700
export EHI=1200


pnback inspecfile=${PN}-fovt.pi inspecoot=${PN}-fovtoot.pi elow=${ELO} ehigh=${EHI} quads="${PNON}" >> pnback_${PN}.log 2>&1 

''')

f.close()

In [ ]:
pnback inspecfile=pnS003-fovt.pi inspecoot=pnS003-fovtoot.pi withplotfiles=yes elow=700 ehigh=1200 quads='yes yes yes yes'  -w 1 -V 10

In [ ]:
rmfgen rmfset=pnS003-0.rmf threshold=1e-06 withenergybins=no energymin=0 energymax=15 nenergybins=30 spectrumset=pnS003-fovt.pi format=var detmaptype=psf detmaparray=detmapfile.ds: withdetbounds=no detxoffset=1200 detyoffset=1200 detxbins=160 detybins=160 withdetbins=yes raweventfile=rawevents.ds correctforpileup=no filterdss=yes withfilteredset=no filteredset=filteredpixellist.ds withrmfset=no psfenergy=2 withsourcepos=no sourcecoords=eqpos sourcex=0 sourcey=0 extendedsource=no modeleffarea=no modelquantumeff=no modelfiltertrans=no modelcontamination=no modelee=yes modelootcorr=no eegridfactor=100 withbadpixcorr=no badpixlocation=notSpecified setbackscale=no keeparfset=yes useodfatt=no ignoreoutoffov=yes crossreg_spectrumset='' crossregionarf=no psfmodel=notSpecified badpixelresolution=2 withbadpixres=no applyxcaladjustment=no acceptchanrange=no applyabsfluxcorr=no  -w 1 -V 4

In [ ]:
rmfgen rmfset=pnS003-0.rmf threshold=1e-06 withenergybins=no energymin=0 energymax=15 nenergybins=30 spectrumset=pnS003-fovt.pi format=var detmaptype=psf detmaparray=detmapfile.ds: withdetbounds=no detxoffset=1200 detyoffset=1200 detxbins=160 detybins=160 withdetbins=yes raweventfile=rawevents.ds correctforpileup=no filterdss=yes withfilteredset=no filteredset=filteredpixellist.ds withrmfset=no psfenergy=2 withsourcepos=no sourcecoords=eqpos sourcex=0 sourcey=0 extendedsource=no modeleffarea=no modelquantumeff=no modelfiltertrans=no modelcontamination=no modelee=yes modelootcorr=no eegridfactor=100 withbadpixcorr=no badpixlocation=notSpecified setbackscale=no keeparfset=yes useodfatt=no ignoreoutoffov=yes crossreg_spectrumset='' crossregionarf=no psfmodel=notSpecified badpixelresolution=2 withbadpixres=no applyxcaladjustment=no acceptchanrange=no applyabsfluxcorr=no  -w 1 -V 4